In [1]:
import sys
sys.version

'3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)]'

In [2]:
pip install face_recognition

Note: you may need to restart the kernel to use updated packages.


In [3]:
import face_recognition

In [4]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [5]:
import face_recognition
import cv2
import os

# Load a sample picture and learn how to recognize it
known_image = face_recognition.load_image_file(r"C:\Users\Asus\Downloads/arpita.jpeg")
known_face_encoding = face_recognition.face_encodings(known_image)[0]

# Extract the name from the image filename (without the extension)
known_face_name = os.path.splitext(os.path.basename(r"C:\Users\Asus\Downloads/arpita.jpeg"))[0]

# Initialize the webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Capture a single frame from the webcam
    ret, frame = video_capture.read()
    
    # Convert the frame from BGR to RGB (OpenCV uses BGR by default)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Find all face locations and face encodings in the current frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Loop through each face found in the current frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces([known_face_encoding], face_encoding)

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        # Use the known face name if there is a match, otherwise label as "Unknown"
        name = known_face_name if matches[0] else "Unknown"

        # Label the face with the name or "Unknown"
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Press 'q' on the keyboard to exit the loop and shut down the camera
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
video_capture.release()
cv2.destroyAllWindows()

In [6]:
import face_recognition
import cv2
import os
import pandas as pd
from datetime import datetime

# Define the path to the folder containing images
images_folder = r"C:\Users\Asus\Downloads\faces"

# Initialize dictionaries to hold face encodings and names
known_face_encodings = []
known_face_names = []

# Load images from the folder and encode faces
for filename in os.listdir(images_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(images_folder, filename)
        image = face_recognition.load_image_file(image_path)
        face_encodings = face_recognition.face_encodings(image)

        for face_encoding in face_encodings:
            known_face_encodings.append(face_encoding)
            known_face_names.append(os.path.splitext(filename)[0])

# Initialize the webcam
video_capture = cv2.VideoCapture(0)

# Create a dictionary to track attendance
attendance = {}

# Define the Excel file path
attendance_file = r"C:\Users\Asus\Downloads\faces\pics.xlsx"

while True:
    # Capture a single frame from the webcam
    ret, frame = video_capture.read()

    # Convert the frame from BGR to RGB (OpenCV uses BGR by default)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Find all face locations and face encodings in the current frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Loop through each face found in the current frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face matches any known faces
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        # Find the name of the matched face
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        # Label the face with the name or "Unknown"
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Record attendance if the recognized name is known and not already recorded today
        if name != "Unknown" and name not in attendance:
            attendance[name] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"Attendance recorded for {name} at {attendance[name]}")

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Press 'q' on the keyboard to exit the loop and shut down the camera
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
video_capture.release()
cv2.destroyAllWindows()

# Save the attendance data to an Excel file
attendance_df = pd.DataFrame(list(attendance.items()), columns=['Name', 'Timestamp'])
attendance_df.to_excel(attendance_file, index=False)
print(f"Attendance data saved to {attendance_file}")

Attendance recorded for IMG-20220628-WA0001 at 2024-07-25 16:48:26
Attendance data saved to C:\Users\Asus\Downloads\faces\pics.xlsx


In [7]:
pip install --user fer

In [8]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [9]:
import face_recognition
import cv2
import os
import pandas as pd
from datetime import datetime
from fer import FER

# Define the path to the folder containing images
images_folder =  r"C:\Users\Asus\Downloads\faces"

# Initialize dictionaries to hold face encodings and names
known_face_encodings = []
known_face_names = []

# Load images from the folder and encode faces
for filename in os.listdir(images_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(images_folder, filename)
        image = face_recognition.load_image_file(image_path)
        face_encodings = face_recognition.face_encodings(image)

        if not face_encodings:
            print(f"No faces found in image: {filename}")
        else:
            for face_encoding in face_encodings:
                known_face_encodings.append(face_encoding)
                known_face_names.append(os.path.splitext(filename)[0])
                print(f"Loaded face encoding for: {filename}")

print(f"Known faces: {known_face_names}")

if not known_face_encodings:
    print("No known face encodings loaded.")

# Initialize the emotion detector
emotion_detector = FER()

# Initialize the webcam
video_capture = cv2.VideoCapture(0)

# Create a dictionary to track attendance
attendance = {}

# Define the Excel file path
attendance_file =r"C:\Users\Asus\Downloads\faces\pics.xlsx"

while True:
    # Capture a single frame from the webcam
    ret, frame = video_capture.read()

    if not ret:
        print("Failed to grab frame from webcam.")
        break

    # Convert the frame from BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Find all face locations and face encodings in the current frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    print(f"Found {len(face_encodings)} faces in the current frame.")

    # Loop through each face found in the current frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face matches any known faces
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)
        name = "Unknown"

        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        # Label the face with the name or "Unknown"
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Detect emotion
        face = frame[top:bottom, left:right]
        emotion, score = emotion_detector.top_emotion(face)
        emotion_text = emotion if emotion else "Unknown"

        # Display the emotion on the frame
        cv2.putText(frame, f"Emotion: {emotion_text}", (left, top - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Record attendance if the recognized name is known and not already recorded today
        if name != "Unknown" and name not in attendance:
            attendance[name] = {
                'Timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                'Emotion': emotion_text
            }
            print(f"Attendance recorded for {name} at {attendance[name]['Timestamp']} with emotion {emotion_text}")

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Press 'q' on the keyboard to exit the loop and shut down the camera
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
video_capture.release()
cv2.destroyAllWindows()

# Save the attendance data to an Excel file
attendance_list = [{'Name': name, 'Timestamp': details['Timestamp'], 'Emotion': details['Emotion']} for name, details in attendance.items()]
attendance_df = pd.DataFrame(attendance_list)
attendance_df.to_excel(attendance_file, index=False)
print(f"Attendance data saved to {attendance_file}")

Loaded face encoding for: IMG-20220628-WA0001.jpg
Loaded face encoding for: IMG-20220708-WA0010.jpg
Loaded face encoding for: IMG-20220709-WA0036.jpg
Loaded face encoding for: Screenshot_2022-12-06-18-28-56-15_99c04817c0de5652397fc8b56c3b3817.jpg
Loaded face encoding for: Snapchat-1011928257.jpg
Loaded face encoding for: Snapchat-1681490918.jpg
Loaded face encoding for: Snapchat-172339934.jpg
Loaded face encoding for: Snapchat-178852411.jpg
Known faces: ['IMG-20220628-WA0001', 'IMG-20220708-WA0010', 'IMG-20220709-WA0036', 'Screenshot_2022-12-06-18-28-56-15_99c04817c0de5652397fc8b56c3b3817', 'Snapchat-1011928257', 'Snapchat-1681490918', 'Snapchat-172339934', 'Snapchat-178852411']
Found 0 faces in the current frame.
Found 1 faces in the current frame.
Attendance recorded for IMG-20220628-WA0001 at 2024-07-25 16:49:07 with emotion sad
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 face

In [10]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install comtypes

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install pycaw

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install mediapipe opencv-python numpy pycaw comtypes screen-brightness-control

Note: you may need to restart the kernel to use updated packages.


In [14]:
import cv2
import mediapipe as mp
import numpy as np
import screen_brightness_control as sbc
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# Initialize Mediapipe and Pycaw
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Set up Pycaw for volume control
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# Get the volume range
vol_range = volume.GetVolumeRange()
min_vol = vol_range[0]
max_vol = vol_range[1]

def calculate_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

def volume_control_from_distance(distance, min_distance, max_distance):
    # Normalize the distance
    norm_distance = np.clip((distance - min_distance) / (max_distance - min_distance), 0, 1)
    # Map the normalized distance to the volume range
    return norm_distance * (max_vol - min_vol) + min_vol

def brightness_control_from_distance(distance, min_distance, max_distance):
    # Normalize the distance
    norm_distance = np.clip((distance - min_distance) / (max_distance - min_distance), 0, 1)
    # Map the normalized distance to the brightness range (0-100)
    return int(norm_distance * 100)

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Set up Mediapipe hand detection
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame")
            break

        # Flip the frame horizontally for a later selfie-view display
        frame = cv2.flip(frame, 1)
        
        # Convert the BGR image to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the frame and find hands
        result = hands.process(rgb_frame)
        
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Draw the hand landmarks on the frame
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                # Get coordinates of the thumb tip (landmark 4) and index finger tip (landmark 8)
                thumb_tip = hand_landmarks.landmark[4]
                index_tip = hand_landmarks.landmark[8]
                
                # Convert normalized coordinates to pixel values
                h, w, _ = frame.shape
                thumb_tip_coords = (int(thumb_tip.x * w), int(thumb_tip.y * h))
                index_tip_coords = (int(index_tip.x * w), int(index_tip.y * h))
                
                # Calculate the distance between thumb and index finger tips
                distance = calculate_distance(thumb_tip_coords, index_tip_coords)
                
                # Map the distance to the volume range and set the system volume
                vol = volume_control_from_distance(distance, 30, 200)  # Adjust min_distance and max_distance as needed
                volume.SetMasterVolumeLevel(vol, None)
                
                # Map the distance to the brightness range and set the screen brightness
                brightness = brightness_control_from_distance(distance, 30, 200)  # Adjust min_distance and max_distance as needed
                sbc.set_brightness(brightness)
                
                # Display the distance, volume, and brightness on the frame
                cv2.putText(frame, f'Distance: {int(distance)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                cv2.putText(frame, f'Volume: {int((vol - min_vol) / (max_vol - min_vol) * 100)}%', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                cv2.putText(frame, f'Brightness: {brightness}%', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
        # Display the frame with the contours drawn
        cv2.imshow('Gesture Volume and Brightness Control', frame)
        
        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '

